In [5]:
# CPU
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
# GPU (CUDA)
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [6]:
# GPU
# !pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu
# CPU
# !pip install llama-cpp-python
# !wget https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.1/llama_cpp_python-0.3.1-cp310-cp310-win_amd64.whl
# !pip install llama_cpp_python-0.3.1-cp310-cp310-win_amd64.whl

In [7]:
# Environment
# !set FORCE_CMAKE=1
# !set CMAKE_ARGS=-DLLAMA_CUBLAS=ON
# !set HF_HUB_OFFLINE = 1
# !set HF_DATASETS_OFFLINE = 1

In [8]:
# Git LFS (Large File Storage)
# !git lfs install

In [9]:
# !pip install transformers huggingface_hub
# !pip install huggingface_hub

In [10]:
# Llama-2-7B-Chat-GGUF
# !git clone https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF ../models/Llama-2-7B-Chat-GGUF

In [11]:
# Llama-2-13b-chat-GGUF
# !git clone https://huggingface.co/TheBloke/Llama-2-13b-chat-GGUF ../models/Llama-2-13b-chat-GGUF

In [ ]:
# Mixtral-8x22B-v0.1.IQ1_M.gguf
#!wget "https://huggingface.co/MaziyarPanahi/Mixtral-8x22B-v0.1-GGUF/resolve/main/Mixtral-8x22B-v0.1.IQ1_M.gguf" -O ../models/Mixtral-8x22B-v0.1.IQ1_M.gguf

In [12]:
import os
import logging
import requests
import pandas as pd
from tqdm.autonotebook import tqdm
from typing import Tuple
# Langchain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
#from langchain_core.embeddings import Embeddings
from langchain.schema.runnable import RunnablePassthrough
# Embeddings
from sentence_transformers import SentenceTransformer
# Chroma
import chromadb
from langchain_chroma import Chroma
from chromadb import PersistentClient, Settings

In [13]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
from config import load_config
config = load_config()
CSV_CLEANED_PATH = config["CSV_CLEANED_PATH"]

In [ ]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
def load_csv(file_path: str) -> pd.DataFrame:
    """Charge le fichier CSV et retourne un DataFrame."""
    try:
        return pd.read_csv(file_path, sep=';')
    except Exception as e:
        logging.error(f"Erreur lors du chargement du fichier CSV : {e}")
        raise

In [16]:
class EmbeddingWrapper:
    def embed_documents(self, texts: list) -> list:
        """Encode les documents et retourne les embeddings."""
        embeddings = []
        for text in tqdm(texts, desc="Encoding documents"):
            embeddings.append(embeddings_model.encode(text))
        return embeddings
    def embed_query(self, query: str) -> list:
        """Encode une requête et retourne l'embedding."""
        return embeddings_model.encode([query])[0]

In [17]:
# class EmbeddingWrapper:
#     def embed_documents(self, texts):
#         embeddings = []
#         for text in tqdm(texts, desc="Encoding documents"):
#             embeddings.append(embeddings_model.encode(text))
#         return embeddings

#     def embed_query(self, query):
#         return embeddings_model.encode([query])[0]

In [18]:
def get_existing_vector_store(persist_directory: str, collection_name: str) -> Tuple[Chroma, bool, PersistentClient]:
    """Récupère un magasin de vecteurs existant."""
    persistent_client = PersistentClient(path=persist_directory, settings=Settings(anonymized_telemetry=False))
    
    try:
        collection = persistent_client.get_collection(collection_name)
        logging.info("Collection existante chargée.")
        return collection, True, persistent_client
    except Exception:
        logging.warning("Collection inexistante.")
        return None, False, persistent_client

In [19]:
def create_vector_store(data: pd.DataFrame, batch_size: int = 10, persist_directory: str = "../models/chroma_langchain_db", collection_name: str = "project_gutenberg") -> Chroma:
    """Crée un magasin de vecteurs à partir des données."""
    collection, exists, persistent_client = get_existing_vector_store(persist_directory, collection_name)
    
    if not exists:
        logging.info("Création d'une nouvelle collection...")
        collection = persistent_client.create_collection(collection_name)
    else:
        return Chroma(
            client=persistent_client,
            collection_name=collection_name,
            embedding_function=EmbeddingWrapper(),
        )

    existing_ids = set()
    try:
        existing_docs = collection.get()['documents']
        existing_ids = {doc['id'] for doc in existing_docs}
    except Exception as e:
        logging.error(f"Erreur lors de la récupération des documents existants : {e}")

    titles = data['Title'].tolist()
    authors = data['Author'].tolist()
    summaries = data['Summary'].tolist()
    
    logging.info("Génération d'IDs uniques...")
    unique_ids = []
    seen_ids = set()
    
    for idx in range(len(titles)):
        unique_id = idx + 1
        while unique_id in seen_ids or (exists and unique_id in existing_ids):
            unique_id += 1
        unique_ids.append(unique_id)
        seen_ids.add(unique_id)

    logging.info("Ajout ou mise à jour des textes dans le magasin de vecteurs...")
    
    for i in tqdm(range(0, len(summaries), batch_size), desc="Traitement des lots"):
        try:
            metadata = [{'author': authors[j], 'ebook_no': str(data.iloc[j]['EBook-No.'])} for j in range(i, min(i + batch_size, len(summaries)))]
            if collection is not None:
                collection.add(ids=unique_ids[i:i + batch_size], documents=summaries[i:i + batch_size], metadatas=metadata)
            else:
                logging.error("La collection est None, impossible d'ajouter les documents.")
        except Exception as e:
            logging.error(f"Erreur lors de l'ajout ou de la mise à jour des textes au lot {i // batch_size}: {e}")

    logging.info(f"{len(summaries)} textes ajoutés ou mis à jour dans le magasin de vecteurs.")
    
    return Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=EmbeddingWrapper(),
    )

In [ ]:
# import chromadb
# from langchain_chroma import Chroma
# from chromadb import PersistentClient, Settings

# persist_directory = "../models/chroma_langchain_db"
# client = chromadb.PersistentClient(path=persist_directory, settings=Settings(anonymized_telemetry=False))
# collection_name = "project_gutenberg"
# collection = client.get_collection(collection_name)

# def display_collection_documents(collection, num_documents=10):
#     """Affiche les documents d'une collection ainsi que leurs métadonnées et IDs."""
#     documents = collection.get()
#     for i, (doc, meta, doc_id) in enumerate(zip(documents['documents'], documents['metadatas'], documents['ids'])):
#         if i >= num_documents:
#             break
#         print(f"Document {i + 1}:")
#         print(f"  ID: {doc_id}")
#         print(f"  Texte: {doc}")
#         print(f"  Métadonnées: {meta}\n")

# display_collection_documents(collection)

In [21]:
# def create_vector_store(data: pd.DataFrame, batch_size: int = 10) -> Chroma:
#     """Crée un magasin de vecteurs à partir des données ou le charge s'il existe déjà."""
#     persist_directory = "../models/chroma_langchain_db"
#     if os.path.exists(persist_directory):
#         logging.info("Chargement du vector store existant...")
#         vector_store = Chroma(
#             embedding_function=EmbeddingWrapper(),
#             persist_directory=persist_directory
#         )
#         return vector_store
#     titles = data['Title'].tolist()
#     authors = data['Author'].tolist()
#     summaries = data['Summary'].tolist()
#     unique_ids = []
#     seen_ids = set()
#     for title, author in zip(titles, authors):
#         base_id = f"{title}_{author}"
#         unique_id = base_id
#         counter = 1
#         while unique_id in seen_ids:
#             unique_id = f"{base_id}_{counter}"
#             counter += 1
#         unique_ids.append(unique_id)
#         seen_ids.add(unique_id)
#     embedding_function = EmbeddingWrapper()
#     vector_store = Chroma(
#         embedding_function=embedding_function,
#         persist_directory=persist_directory
#     )
#     logging.info("Ajout des textes au magasin de vecteurs par lots...")
#     for i in tqdm(range(0, len(summaries), batch_size), desc="Processing batches"):
#         try:
#             metadata = [{'author': authors[j], 'ebook_no': data.iloc[j]['EBook-No.']} for j in range(i, min(i + batch_size, len(summaries)))]
#             vector_store.add_texts(summaries[i:i + batch_size], ids=unique_ids[i:i + batch_size], metadata=metadata)
#         except Exception as e:
#             logging.error(f"Erreur lors de l'ajout des textes au lot {i // batch_size}: {e}")
#     logging.info(f"{len(summaries)} textes ajoutés au magasin de vecteurs.")
#     return vector_store

In [22]:
# def create_vector_store(data):
#     summaries = data['Summary'].tolist()
#     embedding_function = EmbeddingWrapper()
#     vector_store = Chroma(
#         collection_name="gutenberg_books",
#         embedding_function=embedding_function,
#         persist_directory="../models/chroma_langchain_db",
#         )
#     vector_store.add_texts(summaries)
#     return vector_store

In [32]:
def configure_llama(model_path: str = "../models/Llama-2-13B-Chat-GGUF/llama-2-13b-chat.Q8_0.gguf",
                    n_gpu_layers: int = 40, n_batch: int = 512,
                    temperature: float = 0.7, max_tokens: int = 150) -> LlamaCpp:
    """Configure et retourne le modèle Llama."""
    #model_path = os.path.abspath("../models/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q4_K_M.gguf")
    model_path = os.path.abspath("../models/Llama-2-13B-Chat-GGUF/llama-2-13b-chat.Q8_0.gguf")
    logging.info(f"Loading Llama model from: {model_path}")
    try:
        llm = LlamaCpp(
            model_path=model_path,
            n_gpu_layers=n_gpu_layers,
            n_batch=n_batch,
            temperature=temperature,
            max_tokens=max_tokens,
            verbose=True,
        )
        logging.info("Llama model loaded successfully.")
        return llm
    except Exception as e:
        logging.error(f"Error loading Llama model: {e}")
        raise

In [24]:
def get_author(vector_store: Chroma, title: str, data: pd.DataFrame) -> str:
    """Recherche l'auteur d'un livre par son titre à partir du vector store ou du DataFrame en cas d'échec."""
    logging.info(f"Recherche de l'auteur pour : {title}")
    results = vector_store.query(title, k=1)
    if results and len(results) > 0:
        return results[0]['author']
    # Fallback vers le DataFrame si aucune correspondance trouvée dans le vector store
    logging.info("Aucun résultat trouvé dans le vector store, recherche dans le DataFrame...")
    row = data[data['Title'].str.contains(title, case=False)]
    if not row.empty:
        return row.iloc[0]['Author']
    return "Auteur non trouvé."

In [25]:
def get_subject(vector_store: Chroma, title: str, data: pd.DataFrame) -> str:
    """Retourne le sujet d'un livre par son titre à partir du vector store ou du DataFrame en cas d'échec."""
    logging.info(f"Recherche du sujet pour : {title}")
    results = vector_store.query(title, k=1)
    if results and len(results) > 0:
        return results[0]['subject']
    # Fallback vers le DataFrame si aucune correspondance trouvée dans le vector store
    logging.info("Aucun résultat trouvé dans le vector store, recherche dans le DataFrame...")
    row = data[data['Title'].str.contains(title, case=False)]
    if not row.empty:
        return row.iloc[0]['Subject']
    return "Sujet non trouvé."

In [26]:
def get_characters(vector_store: Chroma, title: str, data: pd.DataFrame) -> str:
    """Extrait les personnages d'un livre par son titre à partir du résumé dans le vector store ou le DataFrame en cas d'échec."""
    logging.info(f"Recherche du/des personnage(s) pour : {title}")
    results = vector_store.query(title, k=1)
    if results and len(results) > 0:
        summary = results[0]['summary']
    else:
        # Fallback vers le DataFrame si aucune correspondance trouvée dans le vector store
        logging.info("Aucun résultat trouvé dans le vector store, recherche dans le DataFrame...")
        row = data[data['Title'].str.contains(title, case=False)]
        if not row.empty:
            summary = row.iloc[0]['Summary']
        else:
            return ["Aucun personnage cité."]
    characters = set(word for word in summary.split() if word.istitle())
    if characters:
        return list(characters)
    else:
        return ["Aucun personnage cité."]


In [27]:
def get_full_text_from_url(vector_store: Chroma, title: str, data: pd.DataFrame) -> str:
    """Récupère le texte complet d'un livre en ligne par son titre en utilisant le vector store ou le DataFrame en cas d'échec."""
    logging.info(f"Recherche du texte en ligne pour : {title}")    
    query_embedding = vector_store.embed(title)
    results = vector_store.query(query_embedding)
    if results and len(results) > 0:
        best_match = results[0]
        ebook_no = best_match.get('metadata', {}).get('ebook_no')
        if ebook_no:
            url = f"https://www.gutenberg.org/files/{ebook_no}/{ebook_no}-0.txt"
            try:
                response = requests.get(url)
                response.raise_for_status()
                return response.text
            except requests.exceptions.RequestException as e:
                logging.error(f"Erreur lors de la récupération du texte : {e}")
                return f"Erreur lors de la récupération du texte : {e}"
    # Fallback vers le DataFrame si aucune correspondance trouvée dans le vector store
    logging.info(f"Titre non trouvé dans le vector store, recherche dans le DataFrame : {title}")
    ebook_no_row = data[data['Title'].str.contains(title, case=False)]
    if not ebook_no_row.empty:
        ebook_no = ebook_no_row.iloc[0]['EBook-No.']
        url = f"https://www.gutenberg.org/files/{ebook_no}/{ebook_no}-0.txt"
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.text
        except requests.exceptions.RequestException as e:
            logging.error(f"Erreur lors de la récupération du texte : {e}")
            return f"Erreur lors de la récupération du texte : {e}"
    return "Livre non trouvé."

In [28]:
tools = {
    "get_author": "Fonction pour trouver l'auteur d'un livre",
    "get_subject": "Fonction pour obtenir le sujet d'un livre",
    "get_characters": "Fonction pour extraire les personnages d'un livre",
    "get_full_text_from_url": "Fonction pour récupérer le texte intégral ou complet d'un livre",
}

In [29]:
def setup_qa_chain(llm: LlamaCpp, vector_store: Chroma) -> RunnableSequence:
    """Mise en place de la chaîne QA."""
    template = """
    Cet agent répond à des questions sur des livres du projet Gutenberg. 
    L'utilisateur a posé la question : '{question}'.
    Outils disponibles : {tools}.
    N'oubli pas l'outil get_full_text_from_url pour récupérer le texte intégral ou complet d'un livre
    Fournir une réponse en une phrase concise, directe, précise sans mentionner les outils ou reformuler la question.
    """
    prompt = PromptTemplate(template=template, input_variables=["question", "tools"])
    retriever = vector_store.as_retriever(k=1)
    qa_chain = RunnableSequence(
        {
            "context": retriever, 
            "question": RunnablePassthrough(),
            "tools": RunnablePassthrough()
        }
        | prompt
        | llm
    )    
    return qa_chain

In [30]:
# def setup_qa_chain(llm, vector_store):
#     template = """Contexte : Cet agent, basé sur un modèle de langage (LLM), 
#     a pour objectif de répondre à des questions sur des livres provenant de Project Gutenberg. 
#     Il est capable d'extraire des informations sur les livres et leurs personnages, 
#     ainsi que d'interagir avec le texte complet d'un livre. 

#     Vous êtes un assistant intelligent et bien informé sur les ouvrages du projet Gutenberg. 
#     L'utilisateur a posé la question suivante : '{question}'. 
#     Vous avez accès aux outils suivants : {tools}. 

#     Voici quelques actions que vous pouvez effectuer :
#     1. Trouver l'auteur du livre.
#     2. Donner le sujet du livre.
#     3. Extraire les personnages du résumé.
#     4. Récupérer le texte complet d'un livre.

#     Question : {question}
#     Réponse : Pour répondre à votre question, examinons d'abord les éléments clés et les détails pertinents. 
#     Voici ce que nous savons :
#     """
#     prompt = PromptTemplate(template=template, input_variables=["question"])
#     retriever = vector_store.as_retriever(k=5)
#     qa_chain = RunnableSequence(
#         {"context": retriever, "question": RunnablePassthrough()}
#         | prompt
#         | llm
#     )    
#     return qa_chain

In [ ]:
data = load_csv(CSV_CLEANED_PATH)
logging.info(f"Colonnes du DataFrame : {data.columns.tolist()}")
llm = configure_llama()
vector_store = create_vector_store(data)
qa_chain = setup_qa_chain(llm, vector_store)

In [34]:
questions = [
        "Qui est l'auteur du livre 'L'Assommoir' ?",
        "Quel sujet est traité dans 'House of Atreus' ?",
        "Qui sont les personnages principaux dans 'Uninhabited House' ?",
        "Quel(s) sont le(s) titre(s) de(s) livres(s) de l'auteur Dickens Charles ?",
        "Peux-tu me donner le texte intégral de 'Blue Bird' ?"
]

In [35]:
def main():
    try:
        logging.info(f"Outils : {list(tools.keys())}")
        for question in questions:
            response = qa_chain.invoke({"question": question, "tools": list(tools.keys())})
            logging.info(f"Outils : {list(tools.keys())}\n")
            logging.info(f"Question : {question}\n")
            logging.info(f"Réponse : {response}\n")
    except Exception as e:
        logging.error(f"Une erreur est survenue : {e}")

In [ ]:
if __name__ == "__main__":
    main()

In [30]:
# # Main
# if __name__ == "__main__":
#     file_path = os.path.join("../data", "gutenberg_cleaned.csv")
#     data = load_csv(file_path)
#     print(data.columns)  
#     llm = configure_llama()
#     vector_store = create_vector_store(data)
#     qa_chain = setup_qa_chain(llm, vector_store)
#     question = "Qui est l'auteur du livre Paradise Lost ?"
#     response = qa_chain.invoke({"question": question})
#     print(response)